In [1]:
import tweepy
import time
import os
import json
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy import stats
import csv
import re
from tqdm import tqdm
import traceback

In [2]:
consumerKey = 'qFXNatPD5C2JFEQPlMTXFUr8x'
consumerSecret = 'VCTI9keIp1mqIKLwpuoApI7HSe5b0SpeUHvcQ676J3SOjuuISM'
accessToken = '1371497692069789697-0tx6gputswEwOMlwGfUy4VKBID5SCg'
accessTokenSecret = 'zSFpItEQObd4PBc7E0PFAvoJApglHJqbcT37TW928ji5P'

In [3]:
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)

In [4]:
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [5]:
def itemgetter(*items):
    if len(items) == 1:
        item = items[0]
        def g(obj):
            return obj[item]
    else:
        def g(obj):
            return tuple(obj[item] for item in items)
    return g


def get_followers(screen_name):
    print('Getting Followers list of', screen_name)
    followers = []
    users = tweepy.Cursor(api.followers, screen_name = screen_name, wait_on_rate_limit = True, count = 5000, lang=['en'])

    for user in users.items():
        try:
            followers.append(user)
        except tweepy.TweepError as e:
            print('Going to sleep: ', e)
            time.sleep(60)
            
    print(f'Fetched number of followers for {screen_name} : {len(followers)}')
    return followers

def get_followers_ids(screen_name):
    print('Getting Followers ids of', screen_name)
    followers_ids = []
    users_id = tweepy.Cursor(api.followers_ids, screen_name = screen_name, wait_on_rate_limit = True, count = 5000, lang=['en'])

    for user_id in users_id.items():
        try:
            followers_ids.append(user_id)
        except tweepy.TweepError as e:
            print('Going to sleep: ', e)
            time.sleep(60)
            
    print(f'Fetched number of followers ids for {screen_name} : {len(followers_ids)}')
    return followers_ids

def get_following_ids(screen_name):
    print('Getting Following ids of', screen_name)
    following_ids = []
    try:
        users_id = tweepy.Cursor(api.friends_ids, screen_name = screen_name, wait_on_rate_limit = True, count = 5000, lang=['en'])
        
        for user_id in users_id.items():
            following_ids.append(user_id)
            
    except tweepy.TweepError as e:
        print(e)
        
    print(f'Fetched number of following ids for {screen_name} : {len(following_ids)}')
    return following_ids


def get_following(screen_name):
    print('Getting Following list of ', screen_name)
    following = []
    users = tweepy.Cursor(api.friends, screen_name = screen_name, wait_on_rate_limit = True, count=5000, lang=['en'])
    for user in users.items():
        try:
            following.append(user)
        except tweepy.TweepError as e:
            print("Going to sleep:", e)
            time.sleep(60)
    print(f'Fetched number of followings for {screen_name} : {len(following)}')          
    return following

def append_to_file(file_name, acc_name):
    with open(file_name, "a") as f:
        f.write(',' + acc_name)

In [6]:
def get_data(save_dir, accounts):
    for acc in accounts:

        save_path = os.path.join(save_dir, acc)

        if not os.path.exists(save_path):
            os.mkdir(save_path)
            
            
        # ACCOUNT DATA
        try:
            acc_data = api.get_user(screen_name = acc, wait_on_rate_limit = True)
            ob = {
                    'ID':acc_data.id,
                    'Screen_Name':acc_data.screen_name,
                    'Description': acc_data.description,
                    'StatusesCount':acc_data.statuses_count,
                    'Follower_Count':acc_data.followers_count,
                    'Following_Count':acc_data.friends_count
                    }
        except Exception as ex:
            print(ex)
            continue

        with open(os.path.join(save_path, 'account.json'), 'w') as f:
            json.dump(ob, f)
        
        # FOLLOWERS DATA
#         data = []
#         followers_ids = get_followers_ids(acc)
#         for i in range(0, len(followers_ids), 100):
#             try:
#                 chunk = followers_ids[i:i+100]
#                 users_chunk = api.lookup_users(user_ids=chunk)
#                 for user in users_chunk:
#                     try:
#                         ob = {
#                         'ID':user.id,
#                         'Screen_Name':user.screen_name,
#                         'Description': user.description,
#                         'StatusesCount':user.statuses_count,
#                         'Follower_Count':user.followers_count,
#                         'Following_Count':user.friends_count
#                     } 
#                         data.append(ob)
#                     except Exception as ex:
#                         print(ex)
#                         pass
#             except:
#                 import traceback
#                 traceback.print_exc()
#                 print('Something went wrong, skipping...')
                
#         with open(os.path.join(save_path, 'followers.json'), 'w') as f:
#             json.dump(data, f)
            
        # FOLLOWING DATA
        data = []
        following_ids = get_following_ids(acc)
        for i in range(0, len(following_ids), 100):
            try:
                chunk = following_ids[i:i+100]
                users_chunk = api.lookup_users(user_ids=chunk)
                for user in users_chunk:
                    try:
                        ob = {
                        'ID':user.id,
                        'Screen_Name':user.screen_name,
                        'Description': user.description,
                        'StatusesCount':user.statuses_count,
                        'Follower_Count':user.followers_count,
                        'Following_Count':user.friends_count
                    } 
                        data.append(ob)
                    except Exception as ex:
                        print(ex)
                        continue
            except Exception as e:
                print(e)
                print('Something went wrong, skipping...')
                continue
                
        with open(os.path.join(save_path, 'following.json'), 'w') as f:
            json.dump(data, f)
            

        # FOLLOWERS DATA
#         data = []
#         followers = get_followers(acc)
#         for user in followers:
#             try:
#                 ob = {
#                         'Follows account': acc,
#                         'ID':user.id,
#                         'Screen_Name':user.screen_name,
#                         'Description': user.description,
#                         'StatusesCount':user.statuses_count,
#                         'Follower_Count':user.followers_count,
#                         'Following_Count':user.friends_count
#                 }
#                 data.append(ob)

#             except Exception as ex:
#                 print(ex)
#                 pass

        
        
        # FOLLOWING DATA
#         data = []
#         following = get_following(acc)
#         for user in following:
#             try:
#                 ob = {
#                         'ID':user.id,
#                         'Screen_Name':user.screen_name,
#                         'Description': user.description,
#                         'StatusesCount':user.statuses_count,
#                         'Follower_Count':user.followers_count,
#                         'Following_Count':user.friends_count
#                 } 
#                 data.append(ob)

#             except Exception as ex:
#                 print(ex)
#                 pass

#         with open(os.path.join(save_path, 'following.json'), 'w') as f:
#             json.dump(data, f)

In [7]:
with open(r'C:\Users\psrub\Documents\Python\Twitter\files\base_anti_scientific_accounts.txt', 'r') as f:
    anti_accounts = f.read().split(',')

with open(r'C:\Users\psrub\Documents\Python\Twitter\files\hand_picked_new_anti_hubs.txt', 'r') as f:
    hand_picked_new_anti_hubs = f.read().split(',')
    
with open(r'C:\Users\psrub\Documents\Python\Twitter\files\base_pro_scientific_accounts.txt', 'r') as f:
    pro_accounts = f.read().split(',')

with open(r'C:\Users\psrub\Documents\Python\Twitter\files\hand_picked_new_pro_hubs.txt', 'r') as f:
    hand_picked_new_pro_hubs = f.read().split(',')

In [11]:
save_folder = r'E:\Twitter_data\AUTOMATED_ITERATION_3\base_anti_hubs'
get_data(save_folder, anti_accounts)

Getting Following ids of @RobertKennedyJr
Fetched number of following ids for @RobertKennedyJr : 638
Getting Following ids of @mercola
Fetched number of following ids for @mercola : 28583
[{'code': 63, 'message': 'User has been suspended.'}]
Getting Following ids of @DrButtar
Fetched number of following ids for @DrButtar : 44
Getting Following ids of @unhealthytruth
Fetched number of following ids for @unhealthytruth : 2088
Getting Following ids of @sayerjigmi
Fetched number of following ids for @sayerjigmi : 951
Getting Following ids of @kellybroganmd
Fetched number of following ids for @kellybroganmd : 536
Getting Following ids of @DrChrisNorthrup
Fetched number of following ids for @DrChrisNorthrup : 574
[{'code': 63, 'message': 'User has been suspended.'}]
Getting Following ids of @kevdjenkins1
Fetched number of following ids for @kevdjenkins1 : 483
Getting Following ids of @delbigtree
Fetched number of following ids for @delbigtree : 72
Getting Following ids of @FlatEarthEffect
Fe

Rate limit reached. Sleeping for: 395


Fetched number of following ids for @AnnleeElisha : 4012
Getting Following ids of @FlatArthur
Fetched number of following ids for @FlatArthur : 4239
Getting Following ids of @FlatEarthBaba
Fetched number of following ids for @FlatEarthBaba : 1109
Getting Following ids of @flat_its
Fetched number of following ids for @flat_its : 1770
Getting Following ids of @earthisaplane7
Fetched number of following ids for @earthisaplane7 : 1022
Getting Following ids of @FlatSmacker
Fetched number of following ids for @FlatSmacker : 117
Getting Following ids of @Lumineuse72
Fetched number of following ids for @Lumineuse72 : 1273
Getting Following ids of @flatearthohio
Fetched number of following ids for @flatearthohio : 4275
Getting Following ids of @dmarble
Fetched number of following ids for @dmarble : 27
[{'code': 63, 'message': 'User has been suspended.'}]
Getting Following ids of @facebones777
Fetched number of following ids for @facebones777 : 3820
Getting Following ids of @flatearthaddict
Fetc

Rate limit reached. Sleeping for: 465


Fetched number of following ids for @Gylauer : 648
Getting Following ids of @Spacehehehe
Fetched number of following ids for @Spacehehehe : 1396
[{'code': 50, 'message': 'User not found.'}]
Getting Following ids of @FlatEarthTshirt
Fetched number of following ids for @FlatEarthTshirt : 933
Getting Following ids of @DSL1912
Fetched number of following ids for @DSL1912 : 3313
Getting Following ids of @FlatEarthCity
Fetched number of following ids for @FlatEarthCity : 2527
Getting Following ids of @FlatEarthRT
Fetched number of following ids for @FlatEarthRT : 400
Getting Following ids of @QMFlatEarth
Fetched number of following ids for @QMFlatEarth : 14
Getting Following ids of @FlatAwakening
Fetched number of following ids for @FlatAwakening : 338
Getting Following ids of @FlatEarthOrg
Fetched number of following ids for @FlatEarthOrg : 266
Getting Following ids of @TheFlatEartherr
Fetched number of following ids for @TheFlatEartherr : 425
Getting Following ids of @itsBentooLong
Fetched

...

Fetched number of following ids for @naturalhealthbl : 1722
Getting Following ids of @Anastas32451695
Fetched number of following ids for @Anastas32451695 : 1667
Failed to send request: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=60)
Something went wrong, skipping...
Getting Following ids of @MargoCatholic
Fetched number of following ids for @MargoCatholic : 474
Getting Following ids of @STOP5G_STOP5G
Fetched number of following ids for @STOP5G_STOP5G : 521
Getting Following ids of @VAware1986
Fetched number of following ids for @VAware1986 : 3240
Getting Following ids of @HighWireTalk
Fetched number of following ids for @HighWireTalk : 158
[{'code': 63, 'message': 'User has been suspended.'}]
Getting Following ids of @JakeMaverick5
Fetched number of following ids for @JakeMaverick5 : 305
Getting Following ids of @forcedanarchy
Fetched number of following ids for @forcedanarchy : 1831
Getting Following ids of @heru41
Fetched number of following 

Rate limit reached. Sleeping for: 478


Fetched number of following ids for @nocompulsoryvac : 9768
Getting Following ids of @Novaccineforce
Fetched number of following ids for @Novaccineforce : 416
Getting Following ids of @circleofmamas
Fetched number of following ids for @circleofmamas : 526
Getting Following ids of @VaccineResist
Fetched number of following ids for @VaccineResist : 59
[{'code': 63, 'message': 'User has been suspended.'}]
Getting Following ids of @US4MF
Fetched number of following ids for @US4MF : 785
Getting Following ids of @OrganicConsumer
Fetched number of following ids for @OrganicConsumer : 2326
Getting Following ids of @GMOFreeUSA
Fetched number of following ids for @GMOFreeUSA : 14512
Getting Following ids of @vaxxed2
Fetched number of following ids for @vaxxed2 : 20
Getting Following ids of @CaliVaxChoice
Fetched number of following ids for @CaliVaxChoice : 1809
Getting Following ids of @drsimonegold
Fetched number of following ids for @drsimonegold : 116
Getting Following ids of @ritamollerpalma

Rate limit reached. Sleeping for: 428


Fetched number of following ids for @lifebiomedguru : 4618


In [12]:
save_folder = r'E:\Twitter_data\AUTOMATED_ITERATION_3\hand_picked_base_anti_hubs'
get_data(save_folder, hand_picked_new_anti_hubs)

Getting Following ids of Chemtrails_Uk
Fetched number of following ids for Chemtrails_Uk : 2034
Getting Following ids of jonrappoport
Fetched number of following ids for jonrappoport : 14423
Getting Following ids of ChildrensHD
Fetched number of following ids for ChildrensHD : 637
Getting Following ids of drbloem


Rate limit reached. Sleeping for: 624


Fetched number of following ids for drbloem : 102720
Getting Following ids of DrJudyAMikovits
Fetched number of following ids for DrJudyAMikovits : 38
Getting Following ids of LaLaRueFrench75
Fetched number of following ids for LaLaRueFrench75 : 30347
Getting Following ids of PrisonPlanet
Fetched number of following ids for PrisonPlanet : 804
Getting Following ids of RobSchneider
Fetched number of following ids for RobSchneider : 511
Getting Following ids of IngrahamAngle
Fetched number of following ids for IngrahamAngle : 435
Getting Following ids of NotPorC


Rate limit reached. Sleeping for: 456


Fetched number of following ids for NotPorC : 34399
Getting Following ids of rsbellmedia
Fetched number of following ids for rsbellmedia : 2830
Getting Following ids of JeffereyJaxen
Fetched number of following ids for JeffereyJaxen : 683
Getting Following ids of CarlDevitt
Fetched number of following ids for CarlDevitt : 2268
Getting Following ids of picphysicians
Fetched number of following ids for picphysicians : 324
Getting Following ids of Homestead4Honey


...

Fetched number of following ids for Homestead4Honey : 61547
Getting Following ids of DavidWolfe
Fetched number of following ids for DavidWolfe : 765
Getting Following ids of Cernovich
Fetched number of following ids for Cernovich : 3273
[{'code': 63, 'message': 'User has been suspended.'}]
Getting Following ids of Chemtrails_Uk
Fetched number of following ids for Chemtrails_Uk : 2034
Getting Following ids of WeAreWakinUp
Fetched number of following ids for WeAreWakinUp : 6394
Getting Following ids of 82jsmith
Fetched number of following ids for 82jsmith : 2837
Getting Following ids of jeranism
Fetched number of following ids for jeranism : 2587
Getting Following ids of FEcourious
Fetched number of following ids for FEcourious : 2959
Getting Following ids of ItsFlatFolks
Fetched number of following ids for ItsFlatFolks : 600
Getting Following ids of UnlearningLies
Fetched number of following ids for UnlearningLies : 2700
[{'code': 63, 'message': 'User has been suspended.'}]
Getting Foll

Rate limit reached. Sleeping for: 622


Fetched number of following ids for NotPorC : 34399
Getting Following ids of apiscesdream
Fetched number of following ids for apiscesdream : 704
[{'code': 50, 'message': 'User not found.'}]
Getting Following ids of cuttingchains
Fetched number of following ids for cuttingchains : 2457
Getting Following ids of ParisUsa333
Fetched number of following ids for ParisUsa333 : 1866
Getting Following ids of LeeZilch
Fetched number of following ids for LeeZilch : 352
[{'code': 50, 'message': 'User not found.'}]
Getting Following ids of FlatEarthFreddy
Fetched number of following ids for FlatEarthFreddy : 724
Getting Following ids of MaxisicalG
Fetched number of following ids for MaxisicalG : 1279
Getting Following ids of NikonP1111
Not authorized.
Fetched number of following ids for NikonP1111 : 0
[{'code': 50, 'message': 'User not found.'}]
Getting Following ids of dmarble1
Fetched number of following ids for dmarble1 : 2601
[{'code': 63, 'message': 'User has been suspended.'}]
Getting Followi

Rate limit reached. Sleeping for: 174


Fetched number of following ids for SlayckPhoenix : 513
Getting Following ids of earthisflat87
Fetched number of following ids for earthisflat87 : 301
Getting Following ids of SeekingTruth___
Fetched number of following ids for SeekingTruth___ : 184
Getting Following ids of zeteticmisophia
Fetched number of following ids for zeteticmisophia : 779
Getting Following ids of nocurvature
Fetched number of following ids for nocurvature : 502
Getting Following ids of barmanamar1976
Fetched number of following ids for barmanamar1976 : 2942
Getting Following ids of tsnurds
Fetched number of following ids for tsnurds : 3229
Getting Following ids of RichardReichle
Fetched number of following ids for RichardReichle : 3216
Getting Following ids of Tierraplana6
Fetched number of following ids for Tierraplana6 : 1286


In [13]:
save_folder = r'E:\Twitter_data\AUTOMATED_ITERATION_3\base_pro_hubs'
get_data(save_folder, pro_accounts)

Getting Following ids of @neiltyson
Fetched number of following ids for @neiltyson : 39
Getting Following ids of @deborahblum
Fetched number of following ids for @deborahblum : 965
Getting Following ids of @RebeccaSkloot
Fetched number of following ids for @RebeccaSkloot : 1809
Getting Following ids of @RichardWiseman
Fetched number of following ids for @RichardWiseman : 290
Getting Following ids of @vaughanbell
Fetched number of following ids for @vaughanbell : 2091
Getting Following ids of @neuroconscience


Rate limit reached. Sleeping for: 658


Fetched number of following ids for @neuroconscience : 5062
[{'message': 'Over capacity', 'code': 130}]
Something went wrong, skipping...
Getting Following ids of @edyong209
Fetched number of following ids for @edyong209 : 1939
Getting Following ids of @ProfBrianCox
Fetched number of following ids for @ProfBrianCox : 954
Getting Following ids of @RichardDawkins
Fetched number of following ids for @RichardDawkins : 360
Getting Following ids of @bengoldacre
Fetched number of following ids for @bengoldacre : 3387
Getting Following ids of @BadAstronomer
Fetched number of following ids for @BadAstronomer : 500
Getting Following ids of @michiokaku
Fetched number of following ids for @michiokaku : 51
Getting Following ids of @SamHarrisOrg
Fetched number of following ids for @SamHarrisOrg : 662
Getting Following ids of @HansRosling
Fetched number of following ids for @HansRosling : 166
Getting Following ids of @timberners_lee
Fetched number of following ids for @timberners_lee : 693
Getting Fo

Rate limit reached. Sleeping for: 710


Fetched number of following ids for @danariely : 7
Getting Following ids of @EricTopol
Fetched number of following ids for @EricTopol : 542
Getting Following ids of @bgreene
Fetched number of following ids for @bgreene : 27
Getting Following ids of @MarcusduSautoy
Fetched number of following ids for @MarcusduSautoy : 398
[{'message': 'Over capacity', 'code': 130}]
Something went wrong, skipping...
Getting Following ids of @seanmcarroll
Fetched number of following ids for @seanmcarroll : 456
Getting Following ids of @ProfRWinston
Fetched number of following ids for @ProfRWinston : 49
Getting Following ids of @RandomSpaceFact
Fetched number of following ids for @RandomSpaceFact : 214
Getting Following ids of @carolynporco
Fetched number of following ids for @carolynporco : 464
Getting Following ids of @SebastianThrun
Fetched number of following ids for @SebastianThrun : 16
Getting Following ids of @phylogenomics
Fetched number of following ids for @phylogenomics : 10665
Getting Following

Rate limit reached. Sleeping for: 708


Fetched number of following ids for @kinggary : 8424
Getting Following ids of @plutokiller
Fetched number of following ids for @plutokiller : 92
Getting Following ids of @starstryder
Fetched number of following ids for @starstryder : 1150
[{'code': 63, 'message': 'User has been suspended.'}]
Getting Following ids of @DanTGilbert
Fetched number of following ids for @DanTGilbert : 192
Getting Following ids of @danlevitin
Fetched number of following ids for @danlevitin : 104
Getting Following ids of @2020science
Fetched number of following ids for @2020science : 3509
Getting Following ids of @paulbloomatyale
Fetched number of following ids for @paulbloomatyale : 562
Getting Following ids of @social_brains
Fetched number of following ids for @social_brains : 438
Getting Following ids of @SethShostak
Fetched number of following ids for @SethShostak : 21
Getting Following ids of @dgmacarthur
Fetched number of following ids for @dgmacarthur : 1940
Getting Following ids of @JohnAllenPaulos
Fet

Rate limit reached. Sleeping for: 685


Fetched number of following ids for @LabSpaces : 4782
Getting Following ids of @kejames
Fetched number of following ids for @kejames : 3499
Getting Following ids of @mbeisen
Fetched number of following ids for @mbeisen : 14366
Getting Following ids of @neuroconscience
Fetched number of following ids for @neuroconscience : 5062


In [14]:
save_folder = r'E:\Twitter_data\AUTOMATED_ITERATION_3\hand_picked_base_pro_hubs'
get_data(save_folder, hand_picked_new_pro_hubs)

Getting Following ids of davideagleman
Fetched number of following ids for davideagleman : 1239
Getting Following ids of AstroKatie
Fetched number of following ids for AstroKatie : 3557
Getting Following ids of marynmck
Fetched number of following ids for marynmck : 5106
Getting Following ids of MarsCuriosity
Fetched number of following ids for MarsCuriosity : 172
Getting Following ids of mdichristina
Fetched number of following ids for mdichristina : 2538
Getting Following ids of BillGates
Fetched number of following ids for BillGates : 409
Getting Following ids of SpaceX
Fetched number of following ids for SpaceX : 100
Getting Following ids of elonmusk


Rate limit reached. Sleeping for: 390


Fetched number of following ids for elonmusk : 111
Getting Following ids of scifri
Fetched number of following ids for scifri : 16590
Getting Following ids of chriscmooney
Fetched number of following ids for chriscmooney : 4136
Getting Following ids of ScienceMagazine
Fetched number of following ids for ScienceMagazine : 493
Getting Following ids of bmaher
Fetched number of following ids for bmaher : 1537
Getting Following ids of rebeccawatson
Fetched number of following ids for rebeccawatson : 1059
Getting Following ids of BeeBrookshire
Fetched number of following ids for BeeBrookshire : 2398
Getting Following ids of TEDchris
Fetched number of following ids for TEDchris : 855
Getting Following ids of stevenstrogatz
Fetched number of following ids for stevenstrogatz : 2569
Getting Following ids of NAChristakis
Fetched number of following ids for NAChristakis : 758
Getting Following ids of edge
Fetched number of following ids for edge : 137
Getting Following ids of maggiekb1
Fetched num

Rate limit reached. Sleeping for: 482


Fetched number of following ids for matthewherper : 3006
Getting Following ids of BillNye
Fetched number of following ids for BillNye : 133
Getting Following ids of elakdawalla
Fetched number of following ids for elakdawalla : 2819
Getting Following ids of DNLee5
Fetched number of following ids for DNLee5 : 8470
Getting Following ids of aetiology
Fetched number of following ids for aetiology : 5359
Getting Following ids of alexismadrigal
Fetched number of following ids for alexismadrigal : 5675
Getting Following ids of carlzimmer
Fetched number of following ids for carlzimmer : 2322
Getting Following ids of danieldennett
Fetched number of following ids for danieldennett : 70
Getting Following ids of JenLucPiquant
Fetched number of following ids for JenLucPiquant : 876
Getting Following ids of utafrith
Fetched number of following ids for utafrith : 350
Getting Following ids of michaelshermer
Fetched number of following ids for michaelshermer : 199
Getting Following ids of sciencegoddess

Rate limit reached. Sleeping for: 532


Fetched number of following ids for sciencegoddess : 5559
Getting Following ids of NYTScience
Fetched number of following ids for NYTScience : 463
Getting Following ids of KamalaHarris
Fetched number of following ids for KamalaHarris : 732
Getting Following ids of sciam
Fetched number of following ids for sciam : 780
Getting Following ids of AdamRutherford
Fetched number of following ids for AdamRutherford : 2167
Getting Following ids of CT_Bergstrom
Fetched number of following ids for CT_Bergstrom : 2323
Getting Following ids of zeynep
Fetched number of following ids for zeynep : 795
Getting Following ids of virginiahughes
Fetched number of following ids for virginiahughes : 2832
Getting Following ids of David_Dobbs
Fetched number of following ids for David_Dobbs : 1980
Getting Following ids of NASA
Fetched number of following ids for NASA : 182
Getting Following ids of drkiki
Fetched number of following ids for drkiki : 6727
Getting Following ids of NateSilver538
Fetched number of fo

Rate limit reached. Sleeping for: 549


Fetched number of following ids for amy_harmon : 12878


In [ ]:
json_path = r'C:\Users\psrub\Documents\Python\Twitter\pro_scientific\@neiltyson\following.json'
following_df = pd.read_json(json_path)
following_df['target'] = following_df['ID']
following_df['source'] = 123123
display(following_df)

In [ ]:
frames = []

for acc in anti_accounts:
    
    save_folder = r'C:\Users\psrub\Documents\Python\Twitter\anti_scientific'
    save_path = os.path.join(save_folder,acc)

    account_json = os.path.join(save_path, 'account.json')
    following_json = os.path.join(save_path, 'following.json')
    
    account_df = pd.read_json(account_json,orient='index')
    source_id = account_df.iloc[0][0]
    
    following_df = pd.read_json(following_json)
    following_df['target'] = following_df['ID']
    following_df['source'] = source_id
    #display(following_df)
    frames.append(following_df)
    
    #df = pd.DataFrame(columns=['source','target'])
    #df['target'] = following_df['ID']
    #df['source'] = 21639946
    #display(df)
    
#     G = nx.from_pandas_edgelist(following_df, 'source', 'target') 
#     pos = nx.spring_layout(G)
    
#     f, ax = plt.subplots(figsize=(10, 10))
#     plt.style.use('ggplot')
    
#     nodes = nx.draw_networkx_nodes(G, pos,alpha=0.8)
#     nodes.set_edgecolor('k')
#     nx.draw_networkx_labels(G, pos, font_size=8)
#     nx.draw_networkx_edges(G, pos, width=1.0, alpha=0.2)

    
    
big_df = pd.concat(frames,ignore_index=True)
display(big_df)

In [ ]:
# GET DATA BY CHUNKS  - CHYBA STARE


data = []
for i in range(0, len(ids), 100):
    try:
        chunk = ids[i:i+100]
        data.extend(api.lookup_users(user_ids=chunk))
        try:
                ob = {
                        'Follows account': acc,
                        'ID':user.id,
                        'Screen_Name':user.screen_name,
                        'Description': user.description,
                        'StatusesCount':user.statuses_count,
                        'Follower_Count':user.followers_count,
                        'Following_Count':user.friends_count
                }
                data.append(ob)
    except:
        import traceback
        traceback.print_exc()
        print('Something went wrong, skipping...')
        
        
for user in followers:
            try:
                ob = {
                        'Follows account': acc,
                        'ID':user.id,
                        'Screen_Name':user.screen_name,
                        'Description': user.description,
                        'StatusesCount':user.statuses_count,
                        'Follower_Count':user.followers_count,
                        'Following_Count':user.friends_count
                }
                data.append(ob)

            except Exception as ex:
                print(ex)
                pass

In [ ]:
import csv

In [ ]:
## tweets

In [ ]:
alltweets = []

In [ ]:
acc_name = 'FlatEarthRT'

In [ ]:
new_tweets = api.user_timeline(screen_name=acc_name, count=200, include_rts=False, tweet_mode='extended')

In [ ]:
new_tweets

In [ ]:
alltweets.extend(new_tweets)

In [ ]:
oldest = alltweets[-1].id - 1

In [ ]:
while len(new_tweets) > 0:
    print(f"getting tweets before {oldest}")

    #all subsiquent requests use the max_id param to prevent duplicates
    new_tweets = api.user_timeline(screen_name = screen_name, count=200, max_id=oldest, tweet_mode='extended')

    #save most recent tweets
    alltweets.extend(new_tweets)

    #update the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1

    print(f"...{len(alltweets)} tweets downloaded so far")

In [ ]:
alltweets[0].text

In [ ]:
len(alltweets)

In [ ]:
outtweets = [[tweet.id_str, tweet.created_at, tweet.text] for tweet in alltweets]

In [ ]:
with open(f'new_{screen_name}_tweets.csv', 'w', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["id","created_at","text"])
    writer.writerows(outtweets)

In [ ]:
tweets = api.user_timeline(screen_name = screen_name, count=200, tweet_mode='extended')

In [ ]:
len(tweets)

In [ ]:
outtweets = [[tweet.id_str, 
              tweet.created_at, 
              tweet.favorite_count, 
              tweet.retweet_count, 
              tweet.full_text.encode("utf-8").decode("utf-8")] 
             for idx, tweet in enumerate(tweets)]
df = pd.DataFrame(outtweets,columns=["id","created_at","favorite_count","retweet_count", "text"])

In [ ]:
df

In [ ]:
def cleanTxt(text):
    text=re.sub(r'@[A-Za-z0-9]+','',text) ## removing @ mention
    text=re.sub(r'#','',text)             ## removing # symbol
    text=re.sub(r'RT[\s]+','',text)  ## removing RT followed byspace
 
  #df=df[~df.Tweets.str.contains('RT')] --> another way to remove RT
    text=re.sub(r'https?:\/\/\S+','',text) ## removing https
    return text

In [ ]:
df['Tweets']=df['text'].apply(cleanTxt)   ## applying function

In [ ]:
df.iloc[0].text

In [ ]:
df.iloc[0].Tweets

In [ ]:
def cleanTxt(text):
    text=re.sub(r'@[A-Za-z0-9]+','',text) ## removing @ mention
    text=re.sub(r'#','',text)             ## removing # symbol
    text=re.sub(r'RT[\s]+','',text)  ## removing RT followed byspace
 
    text=re.sub(r'https?:\/\/\S+','',text) ## removing https
    return text

def get_account_tweets(acc_name: str, max_tweets=1000):
    
    tweets = api.user_timeline(screen_name=acc_name, count=200, include_rts=True, tweet_mode='extended')
    all_tweets = []
       
    if tweets:
        all_tweets.extend(tweets)
        oldest_id = tweets[-1].id

        while len(tweets) > 0 and len(all_tweets) <= max_tweets:
            tweets = api.user_timeline(screen_name=acc_name, count=200, include_rts=True, max_id=oldest_id-1, tweet_mode='extended')
            all_tweets.extend(tweets)
            oldest_id = all_tweets[-1].id
            
    print(f'Fetched number of tweets for {acc_name} : {len(all_tweets)}')

    final_tweets = [[tweet.id_str, tweet.created_at, tweet.favorite_count, tweet.retweet_count, tweet.lang, tweet.full_text.encode("utf-8").decode("utf-8")] 
             for idx, tweet in enumerate(all_tweets)]
    
    df = pd.DataFrame(final_tweets,columns=["id","created_at","favorite_count","retweet_count", "language", "text"])
    df['clean_text']=df['text'].apply(cleanTxt)
    
    return df

def get_tweets_for_all_accounts(save_dir, accounts):
    
    for acc in tqdm(accounts):

        save_path = os.path.join(save_dir, acc)
        
        if not os.path.exists(save_path):
            os.mkdir(save_path)
            try:
                df = get_account_tweets(acc)
                df.to_json(os.path.join(save_path, 'tweets.json'))

            except tweepy.TweepError as e:
                print('Going to sleep: ', e)
        else:
            continue

In [ ]:
with open(r'C:\Users\psrub\Documents\Python\Twitter\files\anti_scientific_accounts.txt', 'r') as f:
    anti_accounts = f.read().split(',')
    
with open(r'C:\Users\psrub\Documents\Python\Twitter\files\pro_scientific_accounts.txt', 'r') as f:
    pro_accounts = f.read().split(',')

In [ ]:
with open(r'C:\Users\psrub\Documents\Python\Twitter\files\neutral_accounts.txt', 'r') as f:
    neutral_accounts = f.read().split(',')

In [ ]:
get_tweets_for_all_accounts(anti_accounts)

In [ ]:
get_tweets_for_all_accounts(pro_accounts)

In [ ]:
neutral_accounts_dir = r'E:\Twitter_data\BASE_HUBS\neutral_data_2'
get_tweets_for_all_accounts(neutral_accounts_dir, neutral_accounts)

In [ ]:
with open(r'C:\Users\psrub\Documents\Python\Twitter\files\hand_picked_new_anti_hubs.txt', 'r') as f:
    new_hand_picked_anti_hubs = f.read().split(',')
    
with open(r'C:\Users\psrub\Documents\Python\Twitter\files\hand_picked_new_pro_hubs.txt', 'r') as f:
    new_hand_picked_pro_hubs = f.read().split(',')

In [ ]:
anti_save_dir = r'C:\Users\psrub\Documents\Python\Twitter\hand_picked_new_anti_hubs'
pro_save_dir = r'C:\Users\psrub\Documents\Python\Twitter\hand_picked_new_pro_hubs'

In [ ]:
get_tweets_for_all_accounts(anti_save_dir, new_hand_picked_anti_hubs)

In [ ]:
get_tweets_for_all_accounts(pro_save_dir, new_hand_picked_pro_hubs)

In [ ]:
tmp = get_account_tweets('iLama_u')

In [ ]:
tmp